In [ ]:
# Make sure you ran `download-dependencies.sh` from the root of the repository first!
%pip install --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

%pip install --quiet langchain==0.0.249 "transformers>=4.24,<5"

In [ ]:
# Make sure you ran `download-dependencies.sh` from the root of the repository first!
%pip install --no-build-isolation --force-reinstall \
    ./dependencies/awscli-*-py3-none-any.whl \
    ./dependencies/boto3-*-py3-none-any.whl \
    ./dependencies/botocore-*-py3-none-any.whl

%pip install --quiet langchain==0.0.249 "transformers>=4.24,<5"

In [ ]:
%pip install --quiet "pillow>=9.5,<10"

In [ ]:
!pip install streamlit

In [7]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


In [8]:
boto3_bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': '0e0ab435-cdd4-4f3e-9025-de53a9790db4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 13 Sep 2023 02:08:40 GMT',
   'content-type': 'application/json',
   'content-length': '3156',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0e0ab435-cdd4-4f3e-9025-de53a9790db4'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-g1-text-02',
   'modelId': 'amazon.titan-embed-g1-text-02'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/stability.stable-diffusion-xl',
   'modelId': 'stability.stable-diffusion-xl'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-grande-instruct',
   'modelId': 'ai

In [ ]:
from langchain.llms.bedrock import Bedrock

llm = Bedrock(
    model_id="amazon.titan-tg1-large",
    model_kwargs={
        "maxTokenCount": 4096,
        "stopSequences": [],
        "temperature": 0,
        "topP": 1,
    },
    client=boto3_bedrock,
)

In [ ]:
#### New Model
parameters={
        "maxTokenCount":3000,
        "stopSequences":[],
        "temperature":0.7,
        "topP":0.9
        }

In [ ]:
role = "teacher"
subject = "child"
age = "13"
story_flavor = "interesting"
story = "machine learning"
length = "200"

In [ ]:
prompt_data = "I am a " + role +"." + " I have a " + subject + " who is " + age + " years old. Could you please generate a " + story_flavor\
              + " story for them? The story is about " + story + ". Please write less than " + length + " letters."


In [ ]:
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-tg1-large"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
text_answer = response_body.get("results")[0].get("outputText")
print_ww(text_answer.strip())

In [ ]:
concise_text=[]
text_splitter=text_answer.split("\n")
i=0
for each_sentence in text_splitter:
    if len(each_sentence) != 0:
        concise_text.append(each_sentence)
        i=i+1
print(concise_text)

In [ ]:
import os

os.getcwd()

In [ ]:
import base64
import io
import json
import os
import sys
import shutil

# External Dependencies:
import boto3
from PIL import Image
images=()
images_list=[]
negative_prompts = [
    "poorly rendered",
    "ugly",
    "poor illustration",
    "deformed",
    "weird"
]
style_preset = "digital-art" #digital-art photographic cartoon 

i=1
for each_line in concise_text:
    prompt=each_line
    request = json.dumps({
    "text_prompts": (
        [{"text": prompt, "weight": 1.0}]
        + [{"text": negprompt, "weight": -1.0} for negprompt in negative_prompts]
    ),
    "cfg_scale": 5,
    "seed": 5450,
    "steps": 30,
    "style_preset": style_preset,
    })
    modelId = "stability.stable-diffusion-xl"

    try:
        response = boto3_bedrock.invoke_model(body=request, modelId=modelId)
        response_body = json.loads(response.get("body").read())
        print(response_body["result"])
        base_64_img_str = response_body["artifacts"][0].get("base64")
        #print(f"{base_64_img_str[0:80]}...")
        image_name = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
        image_name.save(f"./storyImages/image_{i}.png")
        i=i+1
        images_list.append(i)
    except Exception as e:
        print(f"{each_line} ----- Failed due to error: {e}")
    



In [ ]:
i=1
for text_line in concise_text:
    print(text_line)
    try:
        image_to_show=Image.open(f"storyImages/image_{i}.png")
        display(image_to_show)
    except:
        pass
    i=i+1
cleanup_storyimages()
    

In [3]:
import os,shutil
def cleanup_storyimages():
    folder = './storyImages'
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
cleanup_storyimages()